## Travelling salesperson problem:
Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?

## 1. Importing libraries and Defining constants

In [1]:
import random
COUNT_OF_CITIES = 10
MIN_DISTANCE = 1
MAX_DISTANCE = 100
POPULATION_SIZE = 10
STARTING_CITY = 1
ITERATIONS = 100

## 2. Creating The Initial Set of candidate solutions (population)
Each candidate solution is a list of cities and the order in which they are visited. We randomly create a population of POPULATOIN_SIZE candidate solutions.<br>
On both ends of each candidate solution, the city from which the salesman starts and returns is added.

In [2]:
# TSP using Genetic algorithm
def create_population(cities_with_dis):
    population = []
    #Creating a population of size 5
    for i in range(POPULATION_SIZE):
        chromosome = [0]*(len(cities_with_dis)-1)
        cities = [city for city in range(len(cities_with_dis))]
        # Starting city will be present at the start and end hence it is to be removed
        cities.remove(STARTING_CITY)
        for i in range(len(cities)):
            gene = random.choice(cities)
            cities.remove(gene)
            chromosome[i] = gene
        # The same starting and ending city is appended in the list at start and end
        chromosome = [STARTING_CITY] + chromosome
        chromosome = chromosome + [STARTING_CITY]
        population.append(chromosome)
    return population

## 3. Fitness Function
Fitness is the total distance traveled.<br>
Lower the distance, higher the fitness.

In [3]:
# Fitness evaluation function
def getFitnessForChromosome(cities_with_dis, chromosome):
    # Accumulate the distance travelled
    distance = 0
    for i in range(len(chromosome)-1):
        distance += cities_with_dis[chromosome[i]][chromosome[i+1]]
    # chromosome = [1, 2, 3, 1] and the distance between 1 and 2 is stored at [1][2] index
    return distance
def getFitnessValues(cities_with_dis, pop):
    fitnessValues = []
    for i in range(len(pop)):
        fitnessValues.append(getFitnessForChromosome(cities_with_dis, pop[i]))
    return fitnessValues

## 4. Roulette wheel selection
The roulette wheel is used, which is a circular disc divided into segments, whose area is proportional to the fitness value of the individual.<br> The wheel spins and a pointer points to a segment, selecting the individual.<br>
The probability of selection of an individual is proportional to its fitness value.. i.e., the probability of selecting a higher fitness individual is higher than the lower fitness individual.

In [4]:
# Associate probability for selecting each solution
def selParents(fitnessVals, population):
    # Sum of all the fitness values
    fits = []
    for i in fitnessVals:
        fits.append((max(fitnessVals)+1)-i)
    sum = 0
    for i in fits:
        sum += i
    probs = []
    cumProbs = []
    idx = 0
    for i in fits:
        probs.append(i/sum)
        if idx != 0:
            cumProbs.append(probs[idx] + cumProbs[idx-1])
        else:
            cumProbs.append(probs[idx])
        idx += 1
    # print(probs)
    # print(cumProbs)
    # Generate a random number between 0 and 1
    ran1 = random.uniform(0,1)
    ran2 = random.uniform(0,1)
    # Determine the range of ran1 and ran2
    p1 = -1
    p2 = -1
    for i in range(len(cumProbs)-1):
        if ran1 >= cumProbs[i] and ran1 <= cumProbs[i+1]:
            p1 = i
        if ran2 >= cumProbs[i] and ran2 <= cumProbs[i+1]:
            p2 = i
    return population[p1], population[p2]

## 5. Crossover:
Crossover will generate two offsprings of the two parents selected using rollete wheel selection
the center region is of size 2<br>
'(s_i, e_i]' is copied from the parent into the respective child as it is.<br>

In [5]:
SIZE_OF_CENTER_REGION = 3
def crossover(p1, p2):
    # Creating the children
    c1 = [-1]*(len(p1))
    c2 = [-1]*(len(p2))
    # Copying the static values
    c1[0] = p1[0]
    c1[-1] = p1[-1]
    c2[0] = p2[0]
    c2[-1] = p2[-1]

    #Trimming the ends
    p1 = p1[1:-1]
    p2 = p2[1:-1]
    t1 = [-1]*(len(p1))
    t2 = [-1]*(len(p2))
    
    s_i = random.randint(0, (len(p1)-SIZE_OF_CENTER_REGION)-1)
    e_i = s_i + SIZE_OF_CENTER_REGION
    # print(p1, ' ', p2)
    print("Starting Index:", s_i + 1, '\nEnding Index: ', e_i)  # Here + 1 is due to the fact that starting 1 is removed.

    # The region to be copied as it is, is: (s_i, e_i]
    for i in range(s_i, e_i):
        t1[i] = p1[i]
        t2[i] = p2[i]
    
    # Order Crossover function:
    c_i = e_i
    p_i = e_i
    size = len(t1)
    while c_i != s_i:
        if p2[p_i] not in t1:
            t1[c_i] = p2[p_i]
            c_i += 1
            c_i %= size
        p_i += 1
        p_i %= size
    # for second list
    c_i = e_i
    p_i = e_i
    while c_i != s_i:
        if p1[p_i] not in t2:
            t2[c_i] = p1[p_i]
            c_i += 1
            c_i %= size
        p_i += 1
        p_i %= size
    # Copying to the childs
    c1[1: -1] = t1
    c2[1: -1] = t2
    return c1, c2
     

## 6. Mutation:
Mutation is done by swapping two cities in the path.<br>
Selecting two random indices and swapping the cities at those indices.


In [6]:
def mutate(chromosome):
    # Selecting the two random indices
    idx1 = random.randint(1, len(chromosome)-2)
    idx2 = random.randint(1, len(chromosome)-2)
    # Swapping the values at the indices
    chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]
    return chromosome
def mutation(c1, c2):
    c1 = mutate(c1)
    c2 = mutate(c2)
    return c1, c2

## 7. Replacement using Elitism:
The best individual from the previous generation is selected and replaced with the worst individual of the current generation.<br>
This ensures that the best individual is always present in the population.<br>
Elitism is to keep the best solutions from the current generation.<br>

In [7]:
def replace(cities_with_dis, population, fitnessValues, c1, c2):
    # Find the two least fit chromosomes
    # But the thing is that the min fitness value is the best. So, let's remove the max values
    minFit1 = max(fitnessValues)
    fitnessValues.remove(minFit1)  # Temporarily remove this to get the second max
    minFit2 = max(fitnessValues)
    fitnessValues.append(minFit1)
    # Replace the least fit chromosome
    for i in range(len(population)):
        if getFitnessForChromosome(cities_with_dis, population[i]) == minFit1:
            population[i] = c1
            break
    # Replace the second least fit chromosome
    for i in range(len(population)):
        if getFitnessForChromosome(cities_with_dis, population[i]) == minFit2:
            population[i] = c2
            break
    return population

## Main function: (testing)
The main loop runs for <b>ITERATION</b> iteration.<br>
The population size is <b>POPULATION-SIZE</b>.<br>
The best individual is the one with the lowest distance traveled.<br>
The best individual is printed at the end of the loop.<br>
For testing the code, <b>COUNT-OF-CITIES</b> is set to 10.<br>
And the distance matrix is set to a random matrix of size <b>COUNT-OF-CITIES x COUNT-OF-CITIES</b>.<br>
Each city is a minimum of <b>MIN-DISTANCE</b> and a maximum of <b>MAX-DISTANCE</b> apart from the other city.<br>

In [8]:
def main():
    cities_with_dis = []
    for i in range(COUNT_OF_CITIES):
        cities = [0]*COUNT_OF_CITIES
        for j in range(COUNT_OF_CITIES):
            if i != j:
                cities[j] = random.randint(MIN_DISTANCE, MAX_DISTANCE)
        cities_with_dis.append(cities)
    print("Distance Matrix:\n================\n", cities_with_dis)
    
    # Generate the population
    population = create_population(cities_with_dis)
    # print(population)
    fitnessVsRecord = []
    ## Main loop
    for i in range(ITERATIONS):
        print("Iteration: ", i+1)
        # Get the fitness values
        fitnessValues = getFitnessValues(cities_with_dis, population)
        # print(fitnessValues)
        # Select parents
        p1, p2 = selParents(fitnessValues, population)
        # print(p1, p2)
        # Crossover
        c1, c2 = crossover(p1, p2)
        # print(c1, c2)
        # Mutation
        c1, c2 = mutation(c1, c2)
        # print(c1, c2)
        # Replace
        population = replace(cities_with_dis, population, fitnessValues, c1, c2)
        # print(population)
        # Get the fitness values
        fitnessValues = getFitnessValues(cities_with_dis, population)
        # print(fitnessValues)
        # Record the best fitness value
        fitnessVsRecord.append(min(fitnessValues))
        # Get the best chromosome
        bestChromosome = population[fitnessValues.index(min(fitnessValues))]
        print("Best Chromosome: ", bestChromosome)
        print("Fitness Value: ", min(fitnessValues))
        print("Distance: ", getFitnessForChromosome(cities_with_dis, bestChromosome))
        print("=========================================")
        # Terminate the loop if fitness value is not changing for 5 iterations
        # if len(fitnessVsRecord) > 5:
        #     if fitnessVsRecord[-1] == fitnessVsRecord[-2] == fitnessVsRecord[-3] == fitnessVsRecord[-4] == fitnessVsRecord[-5]:
        #         break
# Calling the main function
if __name__ == "__main__":
    main()

Distance Matrix:
 [[0, 98, 81, 8, 80, 70, 58, 86, 93, 21], [27, 0, 24, 92, 75, 87, 63, 92, 87, 44], [85, 5, 0, 65, 79, 18, 58, 47, 31, 89], [84, 82, 25, 0, 73, 25, 12, 89, 63, 67], [6, 54, 28, 35, 0, 55, 72, 63, 53, 83], [39, 45, 33, 6, 11, 0, 16, 71, 58, 25], [31, 59, 2, 5, 9, 66, 0, 82, 74, 73], [65, 67, 61, 99, 21, 69, 68, 0, 54, 63], [48, 88, 75, 58, 29, 31, 22, 46, 0, 50], [77, 1, 63, 80, 36, 31, 40, 27, 100, 0]]
Iteration:  1
Starting Index: 4 
Ending Index:  6
Best Chromosome:  [1, 0, 8, 7, 2, 5, 6, 4, 3, 9, 1]
Fitness Value:  373
Distance:  373
Iteration:  2
Starting Index: 3 
Ending Index:  5
Best Chromosome:  [1, 0, 8, 7, 2, 5, 6, 4, 3, 9, 1]
Fitness Value:  373
Distance:  373
Iteration:  3
Starting Index: 2 
Ending Index:  4
Best Chromosome:  [1, 0, 8, 7, 2, 5, 6, 4, 3, 9, 1]
Fitness Value:  373
Distance:  373
Iteration:  4
Starting Index: 3 
Ending Index:  5
Best Chromosome:  [1, 0, 8, 7, 2, 5, 6, 4, 3, 9, 1]
Fitness Value:  373
Distance:  373
Iteration:  5
Starting Index: 